In [1]:
#%%capture
%run ../dca/input/Format.ipynb
import ROOT as root
from array import array
import math
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
root.gStyle.SetOptStat(0)

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


Error in <TUnixSystem::FindDynamicLibrary>: input/logo/PHENIXTools/lib/libLogoPainter.so does not exist in /home/yoren/bnl/ROOT/install/lib:.:/home/yoren/bnl/ROOT/install/lib:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/lib/x86_64-linux-gnu/tls/haswell/x86_64:/lib/x86_64-linux-gnu/tls/haswell:/lib/x86_64-linux-gnu/tls/x86_64:/lib/x86_64-linux-gnu/tls:/lib/x86_64-linux-gnu/haswell/x86_64:/lib/x86_64-linux-gnu/haswell:/lib/x86_64-linux-gnu/x86_64:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/usr/lib/x86_64-linux-gnu/tls/haswell/x86_64:/usr/lib/x86_64-linux-gnu/tls/haswell:/usr/lib/x86_64-linux-gnu/tls/x86_64:/usr/lib/x86_64-linux-gnu/tls:/usr/lib/x86_64-linux-gnu/haswell/x86_64:/usr/lib/x86_64-linux-gnu/haswell:/usr/lib/x86_64-linux-gnu/x86_64:/usr/lib/x86_64-linux-gnu:/lib/glibc-hwcaps/x86-64-v3:/lib/glibc-hwcaps/x86-64-v2:/lib/tls/haswell/x86_64:/lib/tls/haswell:/lib

In [2]:
root.gROOT.SetBatch(True)

In [3]:
# ---------------------------
# Input sample + generation settings
# ---------------------------

# Example:
# part index mapping (your list)
part_names = ["pi0","eta","rho->ee","omega","omega->pi0ee","eta'->gamma ee",#0-5
              "phi->ee","phi->etaee","jpsi","psi2s","ccbar","bbbar","qgp"]#6-12
colors = [root.kRed, root.kBlue, root.kGreen+2, root.kGreen+4, root.kMagenta, root.kMagenta+2, root.kCyan, root.kOrange, root.kViolet, root.kAzure, root.kPink, root.kTeal, root.kGray+2, root.kYellow+2, root.kBlack]

file_names = ["star_pi0_gee_hists.root", "star_pi0_gee_hists.root", "star_eta_gee_hists.root", "star_omega_ee_hists.root", "star_omega_pi0ee_hists.root"\
              , "star_phi_ee_hists.root", "star_phi_etaee_hists.root"\
              , "star_jpsi_ee_hists.root", "thermal1_hists.root"]

In [4]:
cock_hist_read_2d = []; n_entries = []
for iFile, fname in enumerate(file_names):
    f = root.TFile.Open("output/hists/" + fname)
    h = f.Get("h2_m_pt")
    h.SetDirectory(root.nullptr)
    h.SetName("h2_m_pt_" + f"_{iFile}")#part_names[iFile])
    cock_hist_read_2d.append(h)
    n_entries.append(f.Get("h_mult_final").GetEntries())

In [5]:
f_200 = root.TFile.Open("../dca/input/Star200.root","READ")
g200 = f_200.Get("Table 13").Get("Graph2D_y1;1")
if not g200:
    g200 = f_200.Get("Table 13").Get("Graph2D_y1")
if not g200:
    raise RuntimeError("Graph2D_y1 not found")

n = g200.GetN()
f_200.Close

gr_200 = root.TGraphErrors(n)
for i in range(n):
    m  = float(g200.GetX()[i])  # mass
    acc = 1.
    dm = float(g200.GetY()[i])  # dmass (usually bin width)
    val= float(g200.GetZ()[i])  # dN/dmass
    val_err = float(g200.GetErrorZ(i))  # error on dN/dmass
    gr_200.SetPoint(i, m, val/acc)
    gr_200.SetPointError(i, 0.5*dm, val_err/acc)  # x-error from bin width

In [6]:
mass_proj = []
for iFile in range(0,len(cock_hist_read_2d)):
    mass_proj.append(cock_hist_read_2d[iFile].ProjectionY("mass_proj_"+str(iFile)))
    mass_proj[-1].Scale(1./0.75 / n_entries[iFile] / mass_proj[-1].GetBinWidth(1) * 2)  

In [7]:
for iFile in range(2,len(mass_proj)-1):
    mass_proj[0].Add(mass_proj[iFile])
mass_hist = mass_proj[0].Clone("mass_hist")

In [8]:
tfile = root.TFile("../recoeff/input/newccbar6.root","READ")
mee_star_hist = tfile.Get("mee_star_src")#mee_star
mee_star_hist = mee_star_hist.ProjectionX(mee_star_hist.GetName()+f"_projX",3,3)
mee_star_hist.SetDirectory(root.nullptr)
mee_phenix_hist = tfile.Get("mee_phenix_eta05_src")
mee_phenix_hist = mee_phenix_hist.ProjectionX(mee_phenix_hist.GetName()+f"_projX",3,3)
mee_phenix_hist.SetDirectory(root.nullptr)
tfile.Close()
mee_star_hist.Scale(1./mee_star_hist.GetBinWidth(1)); mee_phenix_hist.Scale(1./mee_star_hist.GetBinWidth(1))
mee_star_hist.Scale(1./324329759273*0.068**2 * 2 * 1.5 ); mee_phenix_hist.Scale(1./324329759273*0.068**2 * 2 * 1.5 )
mee_star_hist.Scale(1./0.75*292); mee_phenix_hist.Scale(1./0.75*292)

#mass_hist.Add(mee_star_hist)

In [9]:
sim_pt_file_names = ['ccbar_soft_v17.root']
hists_pt_sim = []
hist_select_3D_names = ["hist_pt_orig","inv_mass_dca_gen_5"]
sim_file_path = "../sim/output/fall25/dca/"
for iFile in range(len(sim_pt_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_pt_file_names[iFile], "read")
    hists_pt_sim_file = []
    for ihist in range(len(hist_select_3D_names)):
        hists_pt_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_pt_sim_file[-1].SetDirectory(root.nullptr)
        hists_pt_sim_file[-1].SetName(hists_pt_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_pt_sim.append(hists_pt_sim_file)
    infile.Close()

In [10]:
hists_Phenix_mass_new = hists_pt_sim[0][1].ProjectionY("mass_new_ccbar",1,80,1,25)
hists_Phenix_mass_new.Scale(1./hists_pt_sim[0][1].GetEntries()/771); hists_Phenix_mass_new.Scale(1./hists_Phenix_mass_new.GetBinWidth(1)); hists_Phenix_mass_new.Scale(50.e6/324329759273*0.068**2 * 2. * 1.5 ) 
hists_Phenix_mass_new.Scale(1./0.75*292)

In [11]:
if False:
    for ibin in range(1, mee_star_hist.GetNbinsX()+1):
        corr = 1.0
        mass = mee_star_hist.GetBinCenter(ibin)
        if mee_phenix_hist.GetBinContent(mee_phenix_hist.FindBin(mass)) > 0:
            corr = hists_Phenix_mass_new.GetBinContent(hists_Phenix_mass_new.FindBin(mass)) / mee_phenix_hist.GetBinContent(mee_phenix_hist.FindBin(mass))
        mee_star_hist.SetBinContent(ibin, mee_star_hist.GetBinContent(ibin) * corr)
        mee_star_hist.SetBinError(ibin, mee_star_hist.GetBinError(ibin) * corr)

In [12]:
star200_paper_ccbar_mass_points = array('d', [0.197, 0.304, 0.422, 0.557, 0.679, 0.814, 0.960, 1.118, 1.296, 1.462, 1.644, 1.822, 1.996, 2.162, 2.387, 2.656, 2.857, 3.142])
star200_paper_ccbar_yield_points = array('d', [ 0.000011998139968649307, 0.00001629391012439856, 0.000022616759492228645, 0.000026938893095901753, 0.000026356399145461657, 0.000021649259091974006, 0.000015941590374559995, 0.000010523298721084975, 0.000006505656657695662, 0.000004021891772683256, 0.0000023285662984981964, 0.0000012905044120433, 8.154407395185154e-7, 4.619095719403757e-7, 2.3974349678010785e-7, 1.0446897695733761e-7, 5.789728889061239e-8, 2.5786500337705836e-8])
star200_paper_ccbar_mass_edges = array('d', [])
for i in range(len(star200_paper_ccbar_mass_points)-1):
    star200_paper_ccbar_mass_edges.append(0.5*(star200_paper_ccbar_mass_points[i]+star200_paper_ccbar_mass_points[i+1]))
star200_paper_ccbar_mass_edges.insert(0, star200_paper_ccbar_mass_points[0]-0.1*(star200_paper_ccbar_mass_points[1]-star200_paper_ccbar_mass_points[0]))
star200_paper_ccbar_mass_edges.append(star200_paper_ccbar_mass_points[-1]+0.1*(star200_paper_ccbar_mass_points[-1]-star200_paper_ccbar_mass_points[-2]))
star200_paper_ccbar_hist_orig = root.TH1D("star200_paper_ccbar_hist_orig","star200_paper_ccbar_hist_orig",len(star200_paper_ccbar_mass_edges)-1,star200_paper_ccbar_mass_edges)
for i in range(len(star200_paper_ccbar_mass_points)):
    star200_paper_ccbar_hist_orig.SetBinContent(i+1, star200_paper_ccbar_yield_points[i]*127)

In [13]:
c7 = root.TCanvas("c7","c7",800,600)
root.gPad.SetLogy()
Format_Graph(mee_star_hist,Lcolor=root.kBlack)
mee_star_hist.Draw("HIST")
Format_Graph(mee_phenix_hist,Lcolor=root.kRed)
hists_Phenix_mass_new.Draw("HIST SAME")
Format_Graph(hists_Phenix_mass_new,Lcolor=root.kBlue)
mee_phenix_hist.Draw("HIST SAME")
Format_Graph(star200_paper_ccbar_hist_orig,Lcolor=root.kGreen+2)
star200_paper_ccbar_hist_orig.Draw("HIST SAME")
c7.Draw()

In [14]:
h_src = star200_paper_ccbar_hist_orig
h_dst = mee_star_hist

xmin = h_src.GetXaxis().GetXmin()
xmax = h_src.GetXaxis().GetXmax()

for ibin in range(1, h_dst.GetNbinsX() + 1):
    x = h_dst.GetBinCenter(ibin)

    # clamp to histogram range to avoid 0 outside
    if x <= xmin:
        val = h_src.GetBinContent(1)
    elif x >= xmax:
        val = h_src.GetBinContent(h_src.GetNbinsX())
    else:
        val = h_src.Interpolate(x)

    h_dst.SetBinContent(ibin, val)

In [15]:
mass_proj[-1].Scale(0.0022)

In [16]:
#add to mass_hist mass_proj[-1] and mee_star_hist (second has a different binning)
mass_hist.Add(mass_proj[-1])
for ibin in range(1, mass_hist.GetNbinsX()+1):
    mass_hist.SetBinContent(ibin, mass_hist.GetBinContent(ibin) + mee_star_hist.GetBinContent(mee_star_hist.FindBin(mass_hist.GetBinCenter(ibin))))
#mee_star_hist.Add(mass_proj[-1])

In [21]:
c = root.TCanvas("c2","c2",900,650)
root.gPad.SetLogy()
gr_200.GetYaxis().SetRangeUser(9e-6, 20)
gr_200.SetMarkerStyle(20)
gr_200.SetMarkerColor(root.kGreen+2)
gr_200.SetLineColor(root.kGreen+2)
gr_200.SetTitle("Table 13; mass; dN/dmass")
gr_200.Draw("AP")

Format_Graph(mass_hist,Msize=1,Mcolor=root.kRed, Lwidth=2)

mass_hist.Draw("HIST SAME")
for i in range(1,len(mass_proj)):
    Format_Graph(mass_proj[i],Msize=0,Lcolor=colors[i-1])
    mass_proj[i].Draw("HIST SAME")

Format_Graph(mee_star_hist,Msize=0,Lcolor=root.kOrange+4)
mee_star_hist.Draw("HIST SAME")
c.Draw()
